## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-12-13-15-35 +0000,bbc,Polls close in first election since Gen Z prot...,https://www.bbc.com/news/articles/cy7jdmvn3l0o...
1,2026-02-12-13-15-00 +0000,wsj,British American Tobacco Turns Cautious on Gui...,https://www.wsj.com/business/earnings/british-...
2,2026-02-12-13-14-00 +0000,wsj,Crocs Logs Lower Profit as Sales Fall,https://www.wsj.com/business/retail/crocs-logs...
3,2026-02-12-13-04-44 +0000,bbc,"Don't change rules for migrants already in UK,...",https://www.bbc.com/news/articles/cdjm3ml3vgzo...
4,2026-02-12-13-01-00 +0000,wsj,Base Metals Benefit From Improved Risk Appetite,https://www.wsj.com/finance/commodities-future...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2434/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
203,trump,34
240,epstein,17
431,new,14
949,canada,13
662,iran,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
345,2026-02-11-15-19-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...,126
25,2026-02-12-12-06-00 +0000,wsj,California Gov. Gavin Newsom is on the road ag...,https://www.wsj.com/politics/gavin-newsom-the-...,120
162,2026-02-12-00-44-00 +0000,wsj,The GOP-led House passed a resolution designed...,https://www.wsj.com/politics/policy/gop-led-ho...,108
178,2026-02-11-23-37-50 +0000,nypost,Six House Republicans break with White House t...,https://nypost.com/2026/02/11/us-news/six-hous...,87
265,2026-02-11-19-48-00 +0000,wsj,Trump-Led Trade Deal Review Poses ‘Important R...,https://www.wsj.com/economy/central-banking/tr...,73


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
345,126,2026-02-11-15-19-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...
25,81,2026-02-12-12-06-00 +0000,wsj,California Gov. Gavin Newsom is on the road ag...,https://www.wsj.com/politics/gavin-newsom-the-...
308,71,2026-02-11-17-27-07 +0000,nypost,Ex-Google CEO Eric Schmidt spent New Year’s wi...,https://nypost.com/2026/02/11/business/ex-goog...
224,61,2026-02-11-21-42-00 +0000,wsj,"At a congressional hearing, Attorney General P...",https://www.wsj.com/politics/policy/pam-bondi-...
145,49,2026-02-12-02-15-54 +0000,nypost,12-year-old girl ID’d as victim in Canada scho...,https://nypost.com/2026/02/11/world-news/girl-...
150,48,2026-02-12-02-02-00 +0000,wsj,Demand for healthcare workers is outstripping ...,https://www.wsj.com/economy/jobs/healthcare-jo...
282,45,2026-02-11-18-56-22 +0000,latimes,Warner Bros. Discovery board faces pressure as...,https://www.latimes.com/entertainment-arts/bus...
174,45,2026-02-11-23-49-00 +0000,wsj,The Republican-led House passed legislation th...,https://www.wsj.com/politics/elections/save-ac...
15,36,2026-02-12-12-26-06 +0000,nypost,Key giveaways in Nancy Guthrie case that show ...,https://nypost.com/2026/02/12/us-news/key-give...
237,35,2026-02-11-21-02-35 +0000,bbc,Casey Wasserman to stay as chair of LA 2028 Ol...,https://www.bbc.com/news/articles/crl4nkg11zjo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
